# Project 2: Two Mode Network  
IS620 Web Analytics  
Aaron Palumbo | Partho Banarjee  

In [8]:
import IPython.display as dis
import networkx as nx
import pandas as pd

## Data

We chose the following dataset to work with:

In [7]:
dis.IFrame(src="http://www.boardsandgender.com/data.php",
           width=800, height=400)

Specifically, we will work with the data from Aug. 2011.

## Data Cleaning

First we need to clean this up a bit. Our data file contains two columns, the first identifies a director, the second identifies a board to which that director is associated. We need to 